In [1]:
from fractions import Fraction
import random


# Locations games avec références sans numpy

In [7]:
message_success = "Tout est en norme avec la théorie"  

def lancer_programme_ref():
    """
    Lance un mini-programme en console pour simuler les équilibre de Nash en locations games avec références
    """
    #Demander les informations à l'utilisateur
    c = input("Entrez le côut de déviation c qui doit être compris entre 1/2 et 15 : \n")
    n = input("Sur une échelle de combien voulez-vous testez les références(100 par défaut)?\n")
    nbre_deviations = input("Entrez le nombre de déviations, par défaut il est de 100\nAppuyez sur la touche entrée si vous voulez laisser la valeur par défaut:\n")
    #Réécrire les paramètres selon l'entrée de l'utilisateur
    n= 10 if n=='' else int(n)
    nbre_deviations = 10000 if nbre_deviations=='' else int(nbre_deviations)
    #Lancer la simulation
    tester_equilibres_ref(Fraction(c),n,int(nbre_deviations))
    
def tester_equilibres_ref(c,n=100,nbre_deviations=100):
    r1_list = []
    r2_list = []
    colors = []
    grille =[Fraction(x,n) for x in range(0,n+1)]
    references = []
    for r1 in grille :
        grille_r2 = list(filter(lambda x: x>=r1,grille))
        for r2 in grille_r2:
            references.append([r1,r2])
            #print("\n*******(",r1,"=",float(r1),",",r2,"=",float(r2),")*********")
            nombres_aleatoires=[]
            result_verification  =  verifier_profile_ref(c,[r1,r2],nbre_deviations)
            if result_verification != None and result_verification["result"]==True:
                r1_list.append(r1)
                r2_list.append(r2)
                colors.append(result_verification["col"])
    #print(message_success)
    verifier_profile_ref(c,references,nbre_deviations)
    plt.scatter(r2_list,r1_list,c=colors)
    plt.title("Nuage de points des références")
    plt.show()

In [8]:
def verifier_profile_ref(c,refs,nbres_deviation =  10): 
    """
    Duopole avec reference
        Permet de vérifier la théorie sur l'existence des équilibres de Nash
    Paramètres:
        c: int
            Le côut de déviation, est un entier ou décimal compris entre 0.5 et 15
        refs: list, tuple
            les références des joueurs en décimales ou fraction compris entre 0 et 1 
        nbre_deviation: Représente le nombre de déviation simulé. la valeur par défaut est 10 000
    return: None
    """
    borne1,borne2 = calculs_bornes(c,refs)
    if borne1>=borne2 :
        return verifier_cas1(c,refs,nbres_deviation)
    else:
        return verifier_cas2(c,refs,nbres_deviation)
    

In [9]:
def calculs_bornes (c,references):
    """
    Retourne les bornes de décision d'équilibre de Nash pour les joueurs, ie 
    r1 + 1/4c et r2 - 1/4c
    
    paramètres:
        refs: list, tuple
            les références des joueurs en décimales ou fraction compris entre 0 et 1, ranger dans l'ordre croissant
    return: tuple
        retourne les bornes        
    """
    refs = sorted(references)
    return refs[0] + Fraction(1,4*c),refs[1] - Fraction(1,4*c)

def verifier_cas1(c,refs,nbres_deviation):
    """
    Duopole avec références:
        Permet de vérifier si le profil (1/2,1/2) est un équilibre de Nash ou pas
    Paramètres:
        c: Le côut de déviation, est un entier ou décimal compris entre 0.5 et 15
        refs[list]: les références décimales ou fraction compris entre 0 et 1
        nbre_deviation: Représente le nombre de déviation simulé 
    return: None
    """
    #print("premier cas")
    borne1,borne2 = calculs_bornes(c,refs)
    actions = [Fraction(1,2),Fraction(1,2)]
    if 1/2 >= borne2 and 1/2 <= borne1 :
        #print("paiements_actuels",[float(x)for x in paiements_actuels])
        for i in range(nbres_deviation):
            result_deviation = joueurs_peut_devier_ref(c,actions,refs)
            if result_deviation != False:
                return print(get_message_danger(result_deviation)) 
        #print("Premier cas avec 1/2, 1/2 est dans l'intervalle")
        #print(actions,"est un équilibre potentiel")
        return {"result": True, "col": 'b'}
    else:
        #print("Premier cas avec:  1/2, 1/2 n'est pas dans l'intervalle")
        for i in range(nbres_deviation):
            if joueurs_peut_devier_ref(c,actions,refs) != False:
                #print("(1/2,1/2) n'est pas un équilibre")
                return {"result": True, "col": 'w'}
        print("(1/2,1/2) ne devrait pas être un équilibre ")

        
def verifier_cas2(c,refs,nbres_deviation):
    """
    Duopole avec références:
        Permet de vérifier si le profil des bornes est bien un équilibre de Nash ou pas
    Paramètres:
        c: Le côut de déviation, est un entier ou décimal compris entre 0.5 et 15
        refs[list]: les références décimales ou fraction compris entre 0 et 1
        nbre_deviation: Représente le nombre de déviation simulé 
    return: None
    """
    equilibres = calculs_bornes(c,refs)
    eq1=c*(refs[1]- refs[0])**2 +refs[1] + refs[0] - 1 - Fraction(1,4*c)
    eq2= c*(refs[1] - refs[0])**2 - (refs[1] + refs[0]) + 1 - Fraction(1,4*c)
    if eq1 >=0 and eq2 >= 0:
        for i in range(nbres_deviation):
            result_deviation = joueurs_peut_devier_ref(c,equilibres,refs)
            #Aucune déviation ne doit être trouvée
            if result_deviation != False:
                return get_message_danger(result_deviation)
        #Si aucune déviation n'est trouvée
        #print("deuxième cas avec eq1 et eq2 vérifiées")
        #print(equilibres, "est un équilibre potentiel")
        return {"result": True, "col": 'm'}
    else:
        #print("deuxième cas avec eq1 et eq2 non vérifiées")
        for i in range(nbres_deviation):
            if joueurs_peut_devier_ref(c,equilibres,refs) != False:
                #print(equilibres, "n'est pas un équilibre")
                return {"result": True, "col": 'w'}
        print(equilibres," ne devrait pas être un équilibre, mais c'est le cas\n reférences:",refs)

In [10]:
def paiement_ref(c,actions,refs):
    """
    Calcule le paiement des deux joueurs et renvoie le résultat dans l'ordre des actions
    Paramètres:
        c: int, float
            Le côut de déviation doit compris entre 0.5 et 15
        actions: list, tuple
            le profil d'actions des joueurs
        refs: list, tuple
            les références des joueurs en décimales ou fraction compris entre 0 et 1 
    return: list
        les paiements des joueurs
    """
    
    #Trier les actions des joueurs
    actions_triees= sorted(actions)
    
    #Vérifier si les actions étaient déjà dans le bon ordre
    is_ordered = actions_triees[0] == actions[0]
    
    #Calculer la moyenne des actions
    moyenne = (actions_triees[0]+actions_triees[1])/2 
    
    #calculer le paiement du joueur ayant joué la plus petite action
    paiement_1 = moyenne - c*(actions_triees[0]-refs[0 if is_ordered else 1])**2
    
    #Calculer le paiement du joueur ayant joué la plus grande action
    paiement_2 = 1- moyenne - c*(actions_triees[1]-refs[1 if is_ordered else 0])**2
    
    """
    Selon que la liste était déjà triée ou pas on retourne les paiements selon l'ordre dans lequel 
    les actions fût envoyées
    """
    if is_ordered:
        return [paiement_1,paiement_2]
    else :
        return [paiement_2,paiement_1]


def joueurs_peut_devier_ref(c,actions,refs):
    """
    Retourne une valeur indiquant si un joueur, a une déviation profitable ou non
    Paramètres:
      c: int, float
            Le côut de déviation doit compris entre 0.5 et 15
        actions: list, tuple
            le profil d'actions des joueurs
        refs: list, tuple
            les références des joueurs en décimales ou fraction compris entre 0 et 1 
    return: boolean,dict
        Retourne False, si aucun joueurs n'a de déviation profitable possible
        Un dict, si un joueur a une déviation profitable possible
    """
    #Le paiement des joueurs à l'équilibre
    paiements_equilibre = paiement_ref(c,actions,refs) 
    
    #On tire au hasard un joueur pour dévier
    joueur = random.randint(0,1)

    deviation = Fraction(round(random.uniform(0,1)*10000),10000)

    #On affecte cette déviation au joueur tiré au hasard précédement
    actions_deviation = list(actions)
    
    actions_deviation[joueur] = deviation

    #On calcule le paiement obtenue après la déviation
    paiement_deviation = paiement_ref(c,actions_deviation,refs)[joueur]
    
    #print("paiements ", deviation)
    
    #Si le paiement de déviation est plus grand, alors ce n'est pas un équilibre de Nash
    if paiement_deviation>paiements_equilibre[joueur]:
        return {
            "c":c,
            "refs": refs,
            "joueur": joueur,
            "paiement_equilibre": paiements_equilibre[joueur],
            "reference": refs[joueur],
            "deviation": deviation,
            "actions": actions,
            "paiement_deviation": paiement_deviation}
    return False

    
def get_message_danger(result):
    """
    Message envoyé dans le cas où on trouve une contradictionavec la théorie
    """
    return """
    *************Données**********
    Références:        {result[refs]}
    actions:           {result[actions]}
    côut de défiation: {result[c]}
    ******************************
    Le joueurs {joueur} ayant pour référence {result[reference]} et ayant joué {action_joueur}  peut dévier
    vers: {result[deviation]}
    Son paiement sans déviation est: {result[paiement_equilibre]}
    Son paiement après déviation est: {result[paiement_deviation]}
    """.format(result=result,action_joueur=result["actions"][result["joueur"]],joueur=result["joueur"]+1)

In [11]:
tester_equilibres_ref(2)

(Fraction(1, 8), Fraction(81, 200))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(0, 1), Fraction(53, 100)]
(Fraction(1, 8), Fraction(83, 200))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(0, 1), Fraction(27, 50)]
(Fraction(27, 200), Fraction(71, 200))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(1, 100), Fraction(12, 25)]
(Fraction(27, 200), Fraction(83, 200))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(1, 100), Fraction(27, 50)]
(Fraction(31, 200), Fraction(3, 8))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(3, 100), Fraction(1, 2)]
(Fraction(31, 200), Fraction(83, 200))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(3, 100), Fraction(27, 50)]
(Fraction(31, 200), Fraction(17, 40))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(3, 100), Fraction(11, 20)]
(Fraction(33, 200), F

(1/2,1/2) ne devrait pas être un équilibre 
(1/2,1/2) ne devrait pas être un équilibre 
(1/2,1/2) ne devrait pas être un équilibre 
(1/2,1/2) ne devrait pas être un équilibre 
(1/2,1/2) ne devrait pas être un équilibre 
(1/2,1/2) ne devrait pas être un équilibre 
(1/2,1/2) ne devrait pas être un équilibre 
(Fraction(101, 200), Fraction(103, 200))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(19, 50), Fraction(16, 25)]
(Fraction(101, 200), Fraction(21, 40))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(19, 50), Fraction(13, 20)]
(Fraction(101, 200), Fraction(107, 200))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(19, 50), Fraction(33, 50)]
(Fraction(101, 200), Fraction(109, 200))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(19, 50), Fraction(67, 100)]
(Fraction(101, 200), Fraction(23, 40))  ne devrait pas être un équilibre, mais c'est le cas
 reférences: [Fraction(19

TypeError: can only concatenate list (not "Fraction") to list